In [246]:
pip install datasets

In [247]:
pip install rouge

In [248]:
pip install evaluate

In [292]:
pip install evaluate

In [249]:
pip install bert-extractive-summarizer


In [250]:
pip install nltk

In [251]:
pip install langchain_community

In [252]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import tensorflow as tf
from langchain.text_splitter import CharacterTextSplitter
from datasets import load_dataset
from transformers import T5ForConditionalGeneration,T5Tokenizer


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [253]:
import os
import pandas as pd
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from transformers import AutoModel
from numpy.linalg import norm



In [254]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


Load Data Corpus and Test Data

In [279]:
# Minecraft Domain corpus
MainTrain=pd.read_csv('gdrive/My Drive/MainTrain.csv')
MainTrain.drop(columns='Unnamed: 0', inplace=True)


In [280]:
# Covid Domain corpus
CovidTrain=pd.read_csv('gdrive/My Drive/Covid.csv')
CovidTrain.drop(columns='Unnamed: 0', inplace=True)


In [281]:
# Africa Domain Corpus
AfricaTrain=pd.read_csv('gdrive/My Drive/Africa.csv')
AfricaTrain.drop(columns='Unnamed: 0', inplace=True)
AfricaTrain.drop(columns='Unnamed: 0.1', inplace=True)


In [282]:
# Test data
TestData = pd.read_csv('gdrive/My Drive/TestData.csv')
TestData.drop(columns='Unnamed: 0', inplace=True)

In [277]:
# Collect a Themse Corpus

AfricaThemes = []
CovidThemes = []
MainThemes = []
for i in MainTrain['QTheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace("'",'')
    g = g.replace(']','')
    g = g.split(',')
    for y in g:
      MainThemes.append(y)

for i in MainTrain['ATheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace("'",'')
    g = g.replace(']','')
    g = g.split(',')
    for y in g:
      MainThemes.append(y)

for i in MainTrain['ContextTheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace(']','')
    g = g.replace("'",'')
    g = g.split(',')
    for y in g:
      MainThemes.append(y)

for i in CovidTrain['QTheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace("'",'')
    g = g.replace(']','')
    g = g.split(',')
    for y in g:
      CovidThemes.append(y)

for i in CovidTrain['ATheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace("'",'')
    g = g.replace(']','')
    g = g.split(',')
    for y in g:
      CovidThemes.append(y)

for i in CovidTrain['ContextTheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace(']','')
    g = g.replace("'",'')
    g = g.split(',')
    for y in g:
      CovidThemes.append(y)

for i in AfricaTrain['QTheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace("'",'')
    g = g.replace(']','')
    g = g.split(',')
    for y in g:
      AfricaThemes.append(y)

for i in AfricaTrain['ATheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace("'",'')
    g = g.replace(']','')
    g = g.split(',')
    for y in g:
      AfricaThemes.append(y)

for i in AfricaTrain['ContextTheme']:

  i = i.replace('}]','')
  i = i.replace("'",'')
  i = i.replace(" ",'')
  i   = i.split(';')
  for g in i:
    g = g.replace('}]','')
    g = g.replace('[','')
    g = g.replace(']','')
    g = g.replace("'",'')
    g = g.split(',')
    for y in g:
      AfricaThemes.append(y)


In [283]:
# Load Embedding model
EmbedModel = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

configuration_bert.py:   0%|          | 0.00/8.24k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_bert.py:   0%|          | 0.00/97.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/275M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [284]:
# Load keyphrase model
ThemeModel = pipeline("text2text-generation", model="ankur310794/bart-base-keyphrase-generation-kpTimes")

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


In [285]:
# Load QA model
model_name = "deepset/roberta-base-squad2"
QAmodel = pipeline('question-answering', model="deepset/roberta-base-squad2", tokenizer=model_name)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Device set to use cpu


Agent Architecture

In [286]:



def model_qa(qa,context,QAmodel): # QA model



  QA_input = {
      'question': qa,
      'context': context
  }
  res = QAmodel(QA_input)
  return res['answer']


In [308]:
def Aggregator(ReportChunks,ReportContext,question,num_qa,themes,domain,EmbModel,QAmodel): # Split Context to Chunks, find best and save them


  final_context = ''
  data = Retriver(question,themes,num_qa,domain,EmbModel)
  ReportChunks = []
  ReportContext = []

  if len(data) ==0:

    final_context = 'None'
  else:



    for cont in data['Context']:

      chunks, chunk_size = len(cont), 3000
      chunkslist = [ cont[i:i+chunk_size] for i in range(0, chunks, chunk_size) ]
      for chunk in chunkslist:
        ReportChunks.append(chunk)


    cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))

    score_list = []
    for chunk in ReportChunks:

      embeddings = EmbModel.encode([question, chunk])
      res = cos_sim(embeddings[0], embeddings[1])
      score_list.append(res)

    d = {'score':score_list, 'Chunk': ReportChunks}
    res_data = pd.DataFrame(data=d)

    res_data = res_data.sort_values(by=['score'], ascending=False)
    res_data = res_data.head(5)


    for cont1 in res_data['Chunk']:
      final_context = final_context + cont1






  answer = model_qa(question,final_context,QAmodel)


  return answer,ReportContext,ReportChunks





In [288]:

def check_theme(qa,ThemeModel): #chech user question on theme identification

  res = False
  domain = []
  themes = ThemeModel(qa)
  themes = themes [0]
  themes  = themes ['generated_text']
  themes  = themes.split(';')
  for theme in themes:
    if (theme in AfricaThemes) == True:
      domain.append('Arfica')
      res = True
    if (theme in MainThemes) == True:
      domain.append('Maincraft')
      res = True
    if (theme in CovidThemes) == True:
      domain.append('Covid')
      res = True

  return res,themes,domain




In [289]:


def Retriver(qa,themes,num_qa,domain,EmbModel): #Search relative questions in corpus

  themes1 = []
  question1 = []
  context1 = []
  answer1 = []
  domain1 = []
  match1 = []
  coll_qa1 = []
  new_dataAfr = []
  new_dataMain = []
  new_dataCovid = []


  for th in themes:
    th = th.lower()
    themes1.append(th)

  for dom in domain:

    if dom == 'Africa':
      matchlist = []


      for themeafr in AfricaTrain['QTheme']:
        matchdom = 0

        ithemeafr = themeafr.replace('}]','')
        themeafr = themeafr.replace("'",'')
        themeafr = themeafr.replace(" ",'')
        themeafr   = themeafr.split(';')
        for meta1 in themeafr:
          meta1 = meta1.replace('}]','')
          meta1 = meta1.replace('[','')
          meta1 = meta1.replace("'",'')
          meta1 = meta1.replace(']','')
          meta1 = meta1.split(',')
          for meta2 in meta1:
            meta2 = meta2.lower()

            if (meta2 in themes1) == True:
              matchdom =  matchdom  + 1
        matchlist.append(matchdom)

      new_dataAfr = AfricaTrain.copy()
      new_dataAfr['match'] = matchlist
      new_dataAfr['domain'] = ['Africa'] * len(new_dataAfr)
      new_dataAfr = new_dataAfr.sort_values(by=['match'], ascending=False)
      new_dataAfr = new_dataAfr.head(5)



    elif dom == 'Maincraft':
      matchlist = []

      for themeafr in MainTrain['QTheme']:
        matchdom = 0
        ithemeafr = themeafr.replace('}]','')
        themeafr = themeafr.replace("'",'')
        themeafr = themeafr.replace(" ",'')
        themeafr   = themeafr.split(';')
        for meta1 in themeafr:
          meta1 = meta1.replace('}]','')
          meta1 = meta1.replace('[','')
          meta1 = meta1.replace("'",'')
          meta1 = meta1.replace(']','')
          meta1 = meta1.split(',')
          for meta2 in meta1:
            meta2 = meta2.lower()

            if (meta2 in themes1) == True:

              matchdom =  matchdom  + 1
        matchlist.append(matchdom)

      new_dataMain = MainTrain.copy()
      new_dataMain['match'] = matchlist
      new_dataMain['domain'] = ['Maincraft'] * len(new_dataMain)
      new_dataMain = new_dataMain.sort_values(by=['match'], ascending=False)
      new_dataMain = new_dataMain.head(5)



    elif dom == 'Covid':
      matchlist = []

      for themeafr in CovidTrain['QTheme']:
        matchdom = 0
        ithemeafr = themeafr.replace('}]','')
        themeafr = themeafr.replace("'",'')
        themeafr = themeafr.replace(" ",'')
        themeafr   = themeafr.split(';')
        for meta1 in themeafr:
          meta1 = meta1.replace('}]','')
          meta1 = meta1.replace('[','')
          meta1 = meta1.replace("'",'')
          meta1 = meta1.replace(']','')
          meta1 = meta1.split(',')
          for meta2 in meta1:
            meta2 = meta2.lower()

            if (meta2 in themes1) == True:

              matchdom =  matchdom  + 1
        matchlist.append(matchdom)

      new_dataCovid = CovidTrain.copy()
      new_dataCovid['match'] = matchlist
      new_dataCovid['domain'] = ['Covid'] * len(new_dataCovid)
      new_dataCovid = new_dataCovid.sort_values(by=['match'], ascending=False)
      new_dataCovid = new_dataCovid.head(5)

    else:
      coll_qa1 = []

  if len(new_dataCovid) != 0:
    for cont in new_dataCovid['context']:
      context1.append(cont)
    for qa in new_dataCovid['question']:
      question1.append(qa)
    for answ in new_dataCovid['answer']:
      answer1.append(answ)
    for dom in new_dataCovid['domain']:
      domain1.append(dom)
    for mat in new_dataCovid['match']:
      match1.append(mat)

  if len(new_dataAfr) != 0:
    for cont in new_dataAfr['Context']:
      context1.append(cont)
    for qa in new_dataAfr['question']:
      question1.append(qa)
    for answ in new_dataAfr['answer']:
      answer1.append(answ)
    for dom in new_dataAfr['domain']:
      domain1.append(dom)
    for mat in new_dataAfr['match']:
      match1.append(mat)

  if len(new_dataMain) != 0:
    for cont in new_dataMain['Context']:
      context1.append(cont)
    for qa in new_dataMain['question']:
      question1.append(qa)
    for answ in new_dataMain['answer']:
      answer1.append(answ)
    for dom in new_dataMain['domain']:
      domain1.append(dom)
    for mat in new_dataMain['match']:
      match1.append(mat)


  d = {'Context':context1, 'question': question1,'answer': answer1,'domain':domain1, 'Match': match1}
  result_data = pd.DataFrame(data=d)
  result_data = result_data.sort_values(by=['Match'], ascending=False)




  score = []
  cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
  for question in result_data['question']:
    embeddings = EmbModel.encode([qa, question])
    res = cos_sim(embeddings[0], embeddings[1])
    score.append(res)

  result_data['score'] = score


  result_data = result_data.sort_values(by=['score'], ascending=False)

  if len(result_data) > num_qa:
    result_data = result_data.head(num_qa)


  return result_data



In [298]:
def main(data,num_qa,EmbModel,QAmodel,ThemeModel): #Main Scheme of Our System
  maxSe = max(data['Seq']) + 1
  pred_answer = []
  conver_list = []



  for conver in range(maxSe):

    Dataconver = data.loc[lambda x: (x['Seq'] == conver)  ].reset_index()
    for question in Dataconver['question']:

      check,themes,domain = check_theme(question,ThemeModel)

      if (conver in conver_list) == False:
        ReportChunks = []
        ReportContext = []

      if check == True:
        answer,ReportContext,ReportChunks = Aggregator(ReportChunks,ReportContext,question,num_qa,themes,domain,EmbModel,QAmodel)
        pred_answer.append(answer)

      else:
        answer = "NOT DOMAIN"
        pred_answer.append(answer)

      conver_list.append(conver)





  d = {'Predict':pred_answer, 'answer': data['answer'],'factual':data['factual'],'Domain':data['Domain']}
  result_data = pd.DataFrame(data=d)
  return result_data





In [ ]:
# Test system
Agent_Result = main((TestData),6,EmbedModel,QAmodel,ThemeModel)


Standart RAG

In [307]:
def StandartRAGModel(data,QAmodel,EmbedModel):
  c = -1
  ans_list = []
  ReportChunks = []
  for i in range(len(data['question'])):
    c = c + 1
    print(c)
    qa = data['question'][i]
    for cont1 in MainTrain['Context']:

      chunks, chunk_size = len(cont1), 10000
      chunkslist = [ cont1[i:i+chunk_size] for i in range(0, chunks, chunk_size) ]
      for chunk in chunkslist:
        ReportChunks.append(chunk)

    for cont2 in CovidTrain['context']:

      chunks, chunk_size = len(cont2), 10000
      chunkslist = [ cont2[i:i+chunk_size] for i in range(0, chunks, chunk_size) ]
      for chunk in chunkslist:
        ReportChunks.append(chunk)

    for cont3 in AfricaTrain['Context']:

      chunks, chunk_size = len(cont3), 10000
      chunkslist = [ cont3[i:i+chunk_size] for i in range(0, chunks, chunk_size) ]
      for chunk in chunkslist:
        ReportChunks.append(chunk)


    cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))

    score_list = []
    for chunk in ReportChunks:


      embeddings = EmbedModel.encode([qa, chunk])
      res = cos_sim(embeddings[0], embeddings[1])
      score_list.append(res)

    d = {'score':score_list, 'Chunk': ReportChunks}
    res_data3 = pd.DataFrame(data=d)

    res_data3 = res_data3.sort_values(by=['score'], ascending=False).reset_index()


    ans = model_qa(qa,res_data3['Chunk'][0],QAmodel)
    ans_list.append(ans)

  d = {'Predict':ans_list, 'answer': data['answer'],'factual':data['factual'],'Context':data["Context"]}
  result_data = pd.DataFrame(data=d)
  return result_data

In [306]:
StandartRAG_Result = StandartRAGModel(TestData,QAmodel,EmbedModel)

In [236]:
#Load Result
StandartRAG_Result = pd.read_csv('gdrive/My Drive/StandartRAG_Result.csv')
StandartRAG_Result.drop(columns='Unnamed: 0', inplace=True)
StandartRAG_Result

,Predict,answer,factual,Domain
0,HIV-1 is the most common HIV-1 infection in ch...,Mother-to-child transmission (MTCT) is the mai...,False,Covid
1,I have a URT sample of nasopharyngeal aspirate...,URT sample nasopharyngeal aspirates,False,Covid
2,"PIV, IFV, and ADV are all viruses that are con...","parainfluenza virus (PIV), influenza virus (IF...",False,Covid
3,The USA and China had good performance in the ...,using two different datasets of weekly influen...,False,Covid
4,It will be used worldwide to identify evidence...,will be used globally to help identify evidenc...,False,Covid
...,...,...,...,...
91,human herpesvirus,"large, non-segmented, positive sense and singl...",True,Covid
92,The curve of influenza deaths by age at death ...,The curve of inﬂuenza deaths by age at death h...,True,Covid
93,We cannot discharge a patient purely based on ...,we cannot discharge a patient purely based on ...,True,Covid
94,The answer is no.,remains unknown,True,Covid


In [240]:
#Load Result
Agent_Result = pd.read_csv('gdrive/My Drive/Agent_Result.csv')
Agent_Result.drop(columns='Unnamed: 0', inplace=True)
Agent_Result

,Predict,answer,factual,Domain
0,HIV-1 is the most common HIV-1 infection in ch...,Mother-to-child transmission (MTCT) is the mai...,False,Covid
1,I have a URT sample of nasopharyngeal aspirate...,URT sample nasopharyngeal aspirates,False,Covid
2,"The PIV, IFV, and ADV are all viruses that are...","parainfluenza virus (PIV), influenza virus (IF...",False,Covid
3,The USA and China had good performance in the ...,using two different datasets of weekly influen...,False,Covid
4,It will be used worldwide to identify evidence...,will be used globally to help identify evidenc...,False,Covid
...,...,...,...,...
191,pathogenic New World arenavirus,Jackie Earle Haley,True,Other
192,...,The Pussycat Dolls,True,Other
193,...,Miami Heat,True,Other
194,...,the mouthpiece,True,Other


Evaluation

In [295]:
import evaluate
bleu = evaluate.load("bleu")
from rouge import Rouge
import statistics

rouge = Rouge()


In [245]:
#Calculate Precision Check Block

InCorpusTest = len(TestData.loc[lambda x: (x['Domain'] != 'Other') ].reset_index())


empt = Agent_Result.tail(3).reset_index()
emp = empt['Predict'][0]

InCorpusAgent  = len(Agent_Result.loc[lambda x: (x['Predict'] ==  emp) ].reset_index()) + len(Agent_Result.loc[lambda x: (x['Predict'] == 'NOT DOMAIN' ) ].reset_index())
dif = abs(InCorpusTest - InCorpusAgent)
precisio =((len(TestData)-dif)*100) / len(TestData)
print("Precision:" ,precisio )




Precision: 98.46938775510205


In [ ]:
Agent_Result = Agent_Result.loc[lambda x: (x['Domain'] != 'Other') ].reset_index()


In [229]:
# Metrics for Agent in all testdata
sc = []
for r in range(len(Agent_Result['Predict'])):
  scores = rouge.get_scores(Agent_Result['Predict'][r], Agent_Result['answer'][r])
  scores = scores[0]
  scores = scores['rouge-1']

  sc.append(scores['r'])
scor = mean(sc)
BlueResults = bleu.compute(predictions=Agent_Result['Predict'], references=Agent_Result['answer'])


print('rogue:',scor)
print('bleu:',BlueResults['bleu'])

rogue: 0.6690441713393018
bleu: 0.3896191424902713


In [230]:
# Metrics for StandartRAG in all testdata
sc = []
for r in range(len(StandartRAG_Result['Predict'])):
  scores = rouge.get_scores(StandartRAG_Result['Predict'][r], StandartRAG_Result['answer'][r])
  scores = scores[0]
  scores = scores['rouge-1']

  sc.append(scores['r'])
scor = mean(sc)
BlueResults = bleu.compute(predictions=StandartRAG_Result['Predict'], references=StandartRAG_Result['answer'])


print('rogue:',scor)
print('bleu:',BlueResults['bleu'])

rogue: 0.58098384293198
bleu: 0.38405689172883384


In [231]:
# Metrics for Agent in only aggregate qa
Agent_Result_sum = Agent_Result.loc[lambda x: (x['factual'] == False) ].reset_index()

sc = []
for r in range(len(Agent_Result_sum['Predict'])):
  scores = rouge.get_scores(Agent_Result_sum['Predict'][r], Agent_Result_sum['answer'][r])
  scores = scores[0]
  scores = scores['rouge-1']

  sc.append(scores['r'])
scor = mean(sc)
BlueResults = bleu.compute(predictions=Agent_Result_sum['Predict'], references=Agent_Result_sum['answer'])


print('rogue:',scor)
print('bleu:',BlueResults['bleu'])

rogue: 0.5739158688992253
bleu: 0.4000926017425058


In [232]:
# Metrics for StandartRAG in only aggregate qa
StandartRAG_Result_sum = StandartRAG_Result.loc[lambda x: (x['factual'] == False) ].reset_index()


sc = []
for r in range(len(StandartRAG_Result_sum['Predict'])):
  scores = rouge.get_scores(StandartRAG_Result_sum['Predict'][r],StandartRAG_Result_sum['answer'][r])
  scores = scores[0]
  scores = scores['rouge-1']

  sc.append(scores['r'])
scor = mean(sc)
BlueResults = bleu.compute(predictions=StandartRAG_Result_sum['Predict'], references=StandartRAG_Result_sum['answer'])


print('rogue:',scor)
print('bleu:',BlueResults['bleu'])

rogue: 0.5214111686593177
bleu: 0.3034557819022597


In [233]:
# Metrics for StandartRAG in only factual qa
StandartRAG_Result_fact = StandartRAG_Result.loc[lambda x: (x['factual'] == True) ].reset_index()


sc = []
for r in range(len(StandartRAG_Result_fact['Predict'])):
  scores = rouge.get_scores(StandartRAG_Result_fact['Predict'][r],StandartRAG_Result_fact['answer'][r])
  scores = scores[0]
  scores = scores['rouge-1']

  sc.append(scores['r'])
scor = mean(sc)
BlueResults = bleu.compute(predictions=StandartRAG_Result_fact['Predict'], references=StandartRAG_Result_fact['answer'])


print('rogue:',scor)
print('bleu:',BlueResults['bleu'])


rogue: 0.6484995404409973
bleu: 0.49266076420138466


In [234]:
# Metrics for Agent in only factual qa
Agent_Result_fact = Agent_Result.loc[lambda x: (x['factual'] == True) ].reset_index()


sc = []
for r in range(len(Agent_Result_fact['Predict'])):
  scores = rouge.get_scores(Agent_Result_fact['Predict'][r],Agent_Result_fact['answer'][r])
  scores = scores[0]
  scores = scores['rouge-1']

  sc.append(scores['r'])
scor = mean(sc)
BlueResults = bleu.compute(predictions=Agent_Result_fact['Predict'], references=Agent_Result_fact['answer'])


print('rogue:',scor)
print('bleu:',BlueResults['bleu'])

rogue: 0.7768562474380553
bleu: 0.3764084447938418
